In [1]:
import pandas as pd
dataset = pd.read_csv("insurance_pre.csv")
dataset.head()

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520


In [2]:
dataset= pd.get_dummies(dataset,drop_first=True,dtype=int)
dataset.head()

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0


In [3]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [4]:
features=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
target=dataset[['charges']]

In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
param={'criterion':['squared_error','absolute_error','friedman_mse','poisson'],'splitter':['best','random']}
grid=GridSearchCV(DecisionTreeRegressor(),param,refit=True,verbose=3,n_jobs=-1)
grid.fit(features,target)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'friedman_mse', 'poisson'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [6]:
best=grid.best_params_
best

{'criterion': 'poisson', 'splitter': 'best'}

In [7]:
re=grid.cv_results_
data=pd.DataFrame.from_dict(re)
data

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011419,0.002644,0.004988,0.000804,squared_error,best,"{'criterion': 'squared_error', 'splitter': 'be...",0.719196,0.607137,0.734010,0.691107,0.660543,0.682399,0.045245,7
1,0.010126,0.004289,0.006704,0.002708,squared_error,random,"{'criterion': 'squared_error', 'splitter': 'ra...",0.743060,0.621284,0.727454,0.691887,0.690253,0.694788,0.042022,4
2,0.037330,0.001764,0.005053,0.001399,absolute_error,best,"{'criterion': 'absolute_error', 'splitter': 'b...",0.737915,0.550691,0.728830,0.725946,0.728065,0.694289,0.071916,5
3,0.033157,0.002344,0.003830,0.001669,absolute_error,random,"{'criterion': 'absolute_error', 'splitter': 'r...",0.689589,0.622476,0.704070,0.721909,0.767881,0.701185,0.047362,3
4,0.009445,0.002282,0.005614,0.003406,friedman_mse,best,"{'criterion': 'friedman_mse', 'splitter': 'best'}",0.714113,0.645171,0.727464,0.744984,0.679700,0.702286,0.035691,2
5,0.006635,0.001186,0.004385,0.001008,friedman_mse,random,"{'criterion': 'friedman_mse', 'splitter': 'ran...",0.692861,0.650334,0.639808,0.687726,0.716765,0.677499,0.028428,8
6,0.010539,0.001796,0.004550,0.001568,poisson,best,"{'criterion': 'poisson', 'splitter': 'best'}",0.742659,0.672603,0.754324,0.735430,0.705640,0.722131,0.029529,1
7,0.007434,0.000963,0.005459,0.001530,poisson,random,"{'criterion': 'poisson', 'splitter': 'random'}",0.653139,0.658653,0.708481,0.700224,0.706045,0.685308,0.024227,6


In [9]:
y_pred=grid.predict(features)
y_pred

array([16884.924 ,  1725.5523,  4449.462 , ...,  1629.8335,  2007.945 ,
       29141.3603])

In [10]:
from sklearn.metrics import r2_score
r_score=r2_score(target,y_pred)
r_score

0.998667156135576

In [11]:
import pickle as pk
pk.dump(grid,open("final_model_dt_grid_without_tts.sav",'wb'))